In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os

In [2]:
data_path = '../data/'
df = pd.read_csv(os.path.join(data_path, 'WA_Fn-UseC_-HR-Employee-Attrition.csv'))
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


There are 35 features in total in the dataset about. 
Let's focus on a few of them:
- Age
- Attrition
- Department
- DistanceFromHome
- Education
- EnvironmentSatisfaction
- Gender
- JobSatisfaction
- MaritalStatus
- MonthlyIncome
- OverTime
- PerformanceRating
- RelationshipSatisfaction
- TotalWorkingYears
- YearsAtCompany

In this lab, we will use attrition as our label, to try to predict the attrition status accroding to other attributes. 


In [3]:
to_keep = {'Age', 'Attrition', 'Department','DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'Gender', 'JobSatisfaction', 'MaritalStatus',
           'MonthlyIncome', 'OverTime', 'PerformanceRating', 'RelationshipSatisfaction','TotalWorkingYears','YearsAtCompany'}
to_drop = set(df.columns)-to_keep
df.drop(to_drop, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 15 columns):
Age                         1470 non-null int64
Attrition                   1470 non-null object
Department                  1470 non-null object
DistanceFromHome            1470 non-null int64
Education                   1470 non-null int64
EnvironmentSatisfaction     1470 non-null int64
Gender                      1470 non-null object
JobSatisfaction             1470 non-null int64
MaritalStatus               1470 non-null object
MonthlyIncome               1470 non-null int64
OverTime                    1470 non-null object
PerformanceRating           1470 non-null int64
RelationshipSatisfaction    1470 non-null int64
TotalWorkingYears           1470 non-null int64
YearsAtCompany              1470 non-null int64
dtypes: int64(10), object(5)
memory usage: 172.3+ KB


# Preprocessing

It's good that we don't have any null value. Let's one hot encode the Attrition, Department, Gender, MaritalStatus and Overtime. 

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

to_encode = {'Attrition', 'Department','Gender','MaritalStatus','OverTime'}
encoders = dict()

for atr in to_encode:
    encoders[atr] = LabelEncoder()
    df[atr] = encoders[atr].fit_transform(df[atr] )
    

Then, let's scale the numeric features. 

In [5]:
numerics = set(df.columns) - to_encode
for atr in numerics:
    df[atr] = df[atr].astype(np.float)    
    ss = StandardScaler()
    df[atr] = ss.fit_transform(df[atr].values.reshape(-1, 1))

In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 15 columns):
Age                         1470 non-null float64
Attrition                   1470 non-null int64
Department                  1470 non-null int64
DistanceFromHome            1470 non-null float64
Education                   1470 non-null float64
EnvironmentSatisfaction     1470 non-null float64
Gender                      1470 non-null int64
JobSatisfaction             1470 non-null float64
MaritalStatus               1470 non-null int64
MonthlyIncome               1470 non-null float64
OverTime                    1470 non-null int64
PerformanceRating           1470 non-null float64
RelationshipSatisfaction    1470 non-null float64
TotalWorkingYears           1470 non-null float64
YearsAtCompany              1470 non-null float64
dtypes: float64(10), int64(5)
memory usage: 172.3 KB


In [7]:
grouped_features = [('Age','MaritalStatus'),('Age','Gender'),('Department','MonthlyIncome'),('Department','YearsAtCompany'),('Age','TotalWorkingYears'),
                    ('Education', 'JobSatisfaction'),('Age','DistanceFromHome'),('Department','PerformanceRating','MonthlyIncome'),
                    ('Education', 'JobSatisfaction','RelationshipSatisfaction'),('Department','OverTime','YearsAtCompany'),
                    ('Age','TotalWorkingYears','YearsAtCompany')]

In [8]:
from sklearn.model_selection import train_test_split

# stratified 90/10 train/test split`
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df.Attrition)

y_train = df_train.Attrition
y_test = df_test.Attrition

X_train = df_train.drop('Attrition', axis=1).values
X_test = df_test.drop('Attrition', axis=1).values

print('train', X_train.shape, 'test', X_test.shape)

train (1323, 14) test (147, 14)


# Initial test (Categorical not encoded)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [10]:
categorical_features = list(to_encode)
numerics = list(numerics)

In [11]:
inputs = Input(shape=(X_train.shape[1],))

In [12]:
x = Dense(units=10, activation='relu')(inputs)
predictions = Dense(1,activation='sigmoid')(x)

In [13]:
model = Model(inputs=inputs, outputs=predictions)

In [14]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 14)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                150       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________


In [15]:
from sklearn import metrics as mt
yhat = np.round(model.predict(X_test))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[82 41]
 [19  5]] 0.591836734694


# Sparse Encoded categorical features test only

In [16]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
X_train_ohe = ohe.fit_transform(df_train[categorical_features].values)
X_test_ohe = ohe.transform(df_test[categorical_features].values)

In [17]:
inputs = Input(shape=(X_train_ohe.shape[1],),sparse=True)

x = Dense(units=10, activation='relu')(inputs)
predictions = Dense(1,activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=predictions)

model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(X_train_ohe,y_train, epochs=10, batch_size=50, verbose=0)

yhat = np.round(model.predict(X_test_ohe))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[123   0]
 [ 12  12]] 0.918367346939


# Dense, Sparce and Combined

In [18]:

ohe = OneHotEncoder()
X_train_ohe = ohe.fit_transform(df_train[categorical_features].values)
X_test_ohe = ohe.transform(df_test[categorical_features].values)


X_train_num =  df_train[numerics].values
X_test_num = df_test[numerics].values


inputsSparse = Input(shape=(X_train_ohe.shape[1],),sparse=True)
xSparse = Dense(units=10, activation='relu')(inputsSparse)


inputsDense = Input(shape=(X_train_num.shape[1],),sparse=False)
xDense = Dense(units=10, activation='relu')(inputsDense)

x = concatenate([xSparse, xDense])
predictions = Dense(1,activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and Dense layers
model = Model(inputs=[inputsSparse,inputsDense], outputs=predictions)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 10)           130         input_3[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 10)           110         input_4[0][0]                    
__________________________________________________________________________________________________
concatenat

In [19]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit([X_train_ohe,X_train_num],y_train, epochs=10, batch_size=50, verbose=0)

yhat = np.round(model.predict([X_test_ohe,X_test_num]))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[122   1]
 [ 23   1]] 0.836734693878


# Dense Embeddings and Deep Model

In [20]:
# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []

for col in categorical_features:
    # encode as ints for the embedding
    X_ints_train.append( df_train[col].values )
    X_ints_test.append( df_test[col].values )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32')
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)

# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False))
x = Dense(units=20, activation='relu')(all_inputs[-1])
all_branch_outputs.append( Dense(units=10,activation='relu')(x) )

# merge the branches together
final_branch = concatenate(all_branch_outputs)
final_branch = Dense(units=1,activation='sigmoid')(final_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(X_ints_train + [X_train_num],
        y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
1323/1323 [==============================] - 0s 211us/step - loss: 0.2486 - acc: 0.5903
Epoch 2/10
1323/1323 [==============================] - 0s 28us/step - loss: 0.2199 - acc: 0.6954
Epoch 3/10
1323/1323 [==============================] - 0s 28us/step - loss: 0.1999 - acc: 0.7581
Epoch 4/10
1323/1323 [==============================] - 0s 28us/step - loss: 0.1857 - acc: 0.7785
Epoch 5/10
1323/1323 [==============================] - 0s 28us/step - loss: 0.1751 - acc: 0.7997
Epoch 6/10
1323/1323 [==============================] - 0s 28us/step - loss: 0.1672 - acc: 0.8141
Epoch 7/10
1323/1323 [==============================] - 0s 29us/step - loss: 0.1614 - acc: 0.8224
Epoch 8/10
1323/1323 [==============================] - 0s 28us/step - loss: 0.1567 - acc: 0.8299
Epoch 9/10
1323/1323 [==============================] - 0s 27us/step - loss: 0.1529 - acc: 0.8337
Epoch 10/10
1323/1323 [==============================] - 0s 29us/step - loss: 0.1497 - acc: 0.8360


In [21]:
yhat = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[123   0]
 [ 24   0]] 0.836734693878


# Wide: Adding Crossed Columns

In [22]:
cross_columns = [['Age','MaritalStatus'],['Age','Gender'],['Department','MonthlyIncome'],['Department','YearsAtCompany'],['Age','TotalWorkingYears'],
                    ['Education', 'JobSatisfaction'],['Age','DistanceFromHome'],['Department','PerformanceRating','MonthlyIncome'],
                    ['Education', 'JobSatisfaction','RelationshipSatisfaction'],['Department','OverTime','YearsAtCompany'],
                    ['Age','TotalWorkingYears','YearsAtCompany']]

In [25]:
# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []

for cols in cross_columns:
    # encode as ints for the embedding
    enc = LabelEncoder()
    # create crossed labels
    X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
    X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)
    
    enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
    X_crossed_train = enc.transform(X_crossed_train)
    X_crossed_test = enc.transform(X_crossed_test)
    X_ints_train.append( X_crossed_train )
    X_ints_test.append( X_crossed_test )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32')
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# merge the branches together
final_branch = concatenate(all_branch_outputs)
final_branch = Dense(units=1,activation='sigmoid')(final_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(X_ints_train,
        y_train, epochs=10, batch_size=32, verbose=1)

TypeError: ('sequence item 0: expected str instance, float found', 'occurred at index 1300')